Mulitprocessing è il modulo che permette di creare e gestire più processi in python.
Le classi principali sono:
- Process
- Queue
- Lock

In [1]:
import multiprocessing as mp

In [2]:
print(f"The number of cores is {mp.cpu_count()}")

The number of cores is 8


## Process
permette di creare un processo figlio del processo principale.

Le due funzioni principali sono:
- start() : Il processo parte e ritorna il suo risultato
- join() : Per terminare il processo dopo che ha fatto il suo lavoro (se non usato il processo rimane IDLE a vita)

se si vuole passare degli argomenti al processo bisogna usare la keyword args.

**Si noti come non sempre i processi vengono usati nell'ordine impartito**

In [3]:
def stampa(string="ciao"):
    print(f"La stringa è {string}")

nomi=["pippo", "pluto", "paperino"]

#Istanzio classe process (non so perchè ci vuole virgola in args ma ci vuole)
procs=[mp.Process(target=stampa, args=(str(i),)) for i in range(40)]
for i in procs:
    i.start()
for i in procs:
    i.join()

La stringa è 0
La stringa è 3La stringa è 2La stringa è 4La stringa è 1
La stringa è 5


La stringa è 6
La stringa è 8
La stringa è 7La stringa è 9

La stringa è 10

La stringa è 11
La stringa è 12
La stringa è 13
La stringa è 14
La stringa è 15
La stringa è 16
La stringa è 17

La stringa è 18La stringa è 19
La stringa è 20
La stringa è 21
La stringa è 22
La stringa è 23
La stringa è 24
La stringa è 25
La stringa è 26
La stringa è 27La stringa è 28

La stringa è 29
La stringa è 30
La stringa è 31
La stringa è 32
La stringa è 33
La stringa è 34
La stringa è 35
La stringa è 36
La stringa è 37
La stringa è 38
La stringa è 39


## Queue
Una queue è una struttura dati First in First out (FIFO) process e thread safe che può contenere qualsiasi oggetto python (in realtà solo quelli pickleable).

Questa struttura è molto utile per condividere dati tra i processi soprattutto se passata come paramentro a un Process

Le funzioni principali sono:
- put() : Inserisce un oggetto nella coda
- get() : Preleva un oggetto dalla coda

Nell'esempio qui sotto la versione seriale è più veloce in quanto il task è molto semplice e c'è un overhead enorme nella creazione dei processi e causa un effort più grande dell' operazione stessa

In [4]:
import time
def somma_2(num,queue):
    res=num+2
    queue.put(res)
N=100
nums=[i for i in range(N)]

def parallel_sum():
    queue=mp.Queue()
    procs=[mp.Process(target=somma_2, args=(num,queue)) for num in nums]
    for p in procs:
        p.start()
    for p in procs:
        p.join()

    results=[queue.get() for p in procs]
    print(results)

def serial_sum():
    queue=mp.Queue()
    for i in nums:
        somma_2(i,queue)
    results=[queue.get() for i in nums]
    print(results)

start_time=time.time()
parallel_sum()
print(f"Time for parallel version: {time.time()-start_time}")
start_time=time.time()
serial_sum()
print(f"Time for serial version: {time.time()-start_time}")



[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
Time for parallel version: 0.4316079616546631
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
Time for serial version: 0.0023338794708251953


## Pool
La classe pool può essere usata per runnare una funzione in parallelo con input multipli.

NB: pool.map ritorna e ha come argomento solo iterabili

La differenza tra map e map_async è che map_async non aspetta che tutti i processi finiscano prima di ritornare

In [18]:
# L'esempio di prima può essere effettuato con i pool
#Il numero in pool è il numero di processi
p=mp.Pool(2)
def somma_2_2(num):
    return num+2

results=p.map(somma_2_2,nums)
print(results)
results=p.map_async(somma_2_2,nums)
print(results.get())

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
